In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

In [2]:
housing = fetch_california_housing()

In [3]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)     #get fit parameters for scaling
X_valid = scaler.transform(X_valid)         # scale validation and test test using those parameters
X_test = scaler.transform(X_test)

# To use Grid/Randomized search we need to wrap our Keras Models in objects that mimic regular Scikit-learn Regressors

In [15]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [16]:
# Create a Keras regressor based on the build_model() function
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)     #KerasRegressor object is a thin wrapper around Keras model

# This model can now be used like a regular Scikit-Learn regressor.


C:\Users\viraj\AppData\Local\Temp\ipykernel_3288\741151798.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)     #KerasRegressor object is a thin wrapper around Keras model


In [17]:
# Using RandomizedSearch

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_dist = {
    "n_hidden" : [1, 2],
    "n_neurons" : np.arange(10,50),
    "learning_rate" : reciprocal(3e-4, 3e-2),
}

In [18]:
rnd_search_cv = RandomizedSearchCV(keras_reg, param_dist, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=50,
                    validation_data = (X_valid, y_valid), 
                    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
)

Epoch 1/50


c:\Users\viraj\miniconda3\envs\tfGPU\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


242/242 [==============================] - 1s 2ms/step - loss: 2.2212 - val_loss: 1.0722
Epoch 2/50
242/242 [==============================] - 0s 2ms/step - loss: 0.8428 - val_loss: 0.7468
Epoch 3/50
242/242 [==============================] - 0s 2ms/step - loss: 0.7065 - val_loss: 0.6897
Epoch 4/50
242/242 [==============================] - 0s 2ms/step - loss: 0.6642 - val_loss: 0.6594
Epoch 5/50
242/242 [==============================] - 0s 2ms/step - loss: 0.6372 - val_loss: 0.6373
Epoch 6/50
242/242 [==============================] - 0s 2ms/step - loss: 0.6154 - val_loss: 0.6162
Epoch 7/50
242/242 [==============================] - 1s 2ms/step - loss: 0.5968 - val_loss: 0.5979
Epoch 8/50
242/242 [==============================] - 1s 2ms/step - loss: 0.5803 - val_loss: 0.5867
Epoch 9/50
242/242 [==============================] - 1s 2ms/step - loss: 0.5664 - val_loss: 0.5693
Epoch 10/50
242/242 [==============================] - 1s 2ms/step - loss: 0.5534 - val_loss: 0.5621
Epoch 11/5

c:\Users\viraj\miniconda3\envs\tfGPU\lib\site-packages\sklearn\model_selection\_search.py:979: UserWarning: One or more of the test scores are non-finite: [-0.43219791 -0.45337245 -0.39619129         nan -0.43543088         nan
 -0.31297247 -0.31477902 -0.35826114 -0.46458197]
  warnings.warn(


363/363 [==============================] - 1s 2ms/step - loss: 0.8265 - val_loss: 0.9097
Epoch 2/50
363/363 [==============================] - 1s 2ms/step - loss: 0.5485 - val_loss: 0.4886
Epoch 3/50
363/363 [==============================] - 1s 2ms/step - loss: 0.4536 - val_loss: 0.4368
Epoch 4/50
363/363 [==============================] - 1s 2ms/step - loss: 0.4226 - val_loss: 0.4154
Epoch 5/50
363/363 [==============================] - 1s 2ms/step - loss: 0.4071 - val_loss: 0.4133
Epoch 6/50
363/363 [==============================] - 1s 2ms/step - loss: 0.3942 - val_loss: 0.3936
Epoch 7/50
363/363 [==============================] - 1s 2ms/step - loss: 0.3876 - val_loss: 0.3889
Epoch 8/50
363/363 [==============================] - 1s 2ms/step - loss: 0.3870 - val_loss: 0.3817
Epoch 9/50
363/363 [==============================] - 1s 2ms/step - loss: 0.3731 - val_loss: 0.4145
Epoch 10/50
363/363 [==============================] - 1s 2ms/step - loss: 0.3703 - val_loss: 0.3766
Epoch 11/5

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x00000215BD40E6D0>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000215BD4053D0>,
                                        'n_hidden': [1, 2],
                                        'n_neurons': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49])})

In [19]:
rnd_search_cv.best_params_

{'learning_rate': 0.00817415598352801, 'n_hidden': 2, 'n_neurons': 40}